# 3. LangChain基础

## 3.3. LangChain的回退

### 3.3.1. 处理LLM调用API的错误

In [1]:
from unittest.mock import patch
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
import httpx
from openai import RateLimitError

# 创建一个 RateLimitError 对象
response = httpx.Response(
    status_code=429,                                                       # 429: Too Many Requests 错误码
    request=httpx.Request('GET', 'https://api.openai.com/v1/completions')  # 模拟一个请求对象
)

body = {
    'error': {
        'message': 'Rate limit exceeded.',
        'type': 'rate_limit_error',
        'param': None,
        'code': 'rate_limit_error'
    }
}

error = RateLimitError(message="Rate limit exceeded.", response=response, body=body)
# print(error)

#### 1. 模拟触发速率限制错误

In [2]:
openai_api_key = 'EMPTY'
openai_api_base = 'http://localhost:11434/v1'
openai_llm = ChatOpenAI(openai_api_key=openai_api_key, openai_api_base=openai_api_base, temperature=0, max_tokens=256, max_retries=0)

with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(openai_llm.invoke("你是谁？"))
    except RateLimitError:
        print("遇到错误")

遇到错误


#### 2. 测试回退功能

In [3]:
openai_api_key = 'EMPTY'
openai_api_base = 'http://localhost:11434/v1'
openai_llm = ChatOpenAI(openai_api_key=openai_api_key, openai_api_base=openai_api_base, temperature=0, max_tokens=256, max_retries=0)
qwen_llm = ChatOllama(model="qwen:1.8b")
llm = openai_llm.with_fallbacks([qwen_llm])

with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(llm.invoke("你是谁？"))
    except RateLimitError:
        print("遇到错误")

/tmp/ipykernel_102261/3922159719.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  qwen_llm = ChatOllama(model="qwen:1.8b")


content='我是来自阿里云的大规模语言模型，我叫通义千问。我能够理解和生成多种语言，包括但不限于中文、英文、日文等。\n\n作为阿里云开发的超大规模语言模型，我可以回答各种问题，提供信息和帮助决策；还可以根据用户提出的问题或指令，进行文本创作，比如写故事、写诗歌、写科技论文等；此外，我还能够实现自动翻译、语音合成、聊天机器人等功能，为用户提供更加智能化和便捷化的服务体验。\n\n总之，我是阿里云开发的超大规模语言模型，我具备多种语言理解和生成能力，能够回答各种问题，提供信息和帮助决策；还可以根据用户提出的问题或指令，进行文本创作，比如写故事、写诗歌、写科技论文等；此外，我还能够实现自动翻译、语音合成、聊天机器人等功能，为用户提供更加智能化和便捷化的服务体验。' additional_kwargs={} response_metadata={'model': 'qwen:1.8b', 'created_at': '2025-02-20T09:21:23.70244073Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 44302654905, 'load_duration': 24749125, 'prompt_eval_count': 11, 'prompt_eval_duration': 306000000, 'eval_count': 184, 'eval_duration': 43970000000} id='run-34ecea82-636f-4081-9df4-bcfd9efe8f56-0'


#### 3. 使用具有回退功能的LLM

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你真是一个贴心的助手，每次回复都会附上赞美之词。",
        ),
        ("human", "为什么你喜欢{city}"),
    ]
)

chain = prompt | llm
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"city":"利川"}))
    except RateLimitError:
        print("Hit error")

### 3.3.2. 处理序列回退

### 3.3.3. 处理长输入

### 3.3.4. 回退到更好的模型